In [145]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD_Python_Module import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "n0tan0theruzer123"

# Connect to database via CRUD Module
shelter = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Grazioso Salvare’s logo
# I decoded encoded_image to a string here rather than down below
image_filename = 'Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode('utf-8')

app.layout = html.Div([

# Not sure if you wanted us to uncomment this as a skiplink or not but I left it in here just in case
#    html.Div(id='hidden-div', style={'display':'none'}),
    
    # Logo image and link
    html.Center(
        html.A(
            html.Img(
                alt='Grazioso Salvare Logo',
                src='data:image/png;base64,{}'.format(encoded_image),
                style={'width': '250px', 'height': '200px'}
            ),
            href="https://www.snhu.edu",
            target="_blank"
        )
    ),
    
    html.Hr(),
    
    # A flexbox that has the filter dropdown, dashboard title, and unique identifier in it
    html.Div(
        style={'display' : 'flex' , 'justify-content' : 'space-between', 'align-items' : 'center'},
        children = [
            
            # Dropdown with filter options  
            dcc.Dropdown(
                id="filter-type",
                options=[
                    {"label": "Water Rescue", "value": "Water Rescue"},
                    {"label": "Mountain or Wilderness Rescue", "value": "Mountain or Wilderness Rescue"},
                    {"label": "Disaster Rescue or Individual Tracking", "value": "Disaster Rescue or Individual Tracking"},
                    {"label": "Reset (All Records)", "value": "Reset"}
                ],
                value="Reset",
                clearable=False,
                style={'width': '300px', "flex": '1 1 300px'}
            ),
            
             # Title of dashboard
            html.B(
                html.H2('Animal Rescue Dashboard', style={"text-align": "center"}), 
                style={"flex": "1 1 auto"}
            ),
            
            # Unique Identifier
            dcc.Markdown('''
                `Created By Global Rain's: Nathaniel McCarthy`
            ''', style={
                    "text-align": "right",
                    "flex": "1 1 auto"
            })

        ]
    ),
    
    html.Hr(),
    
    # Data table
    dash_table.DataTable(
        id='datatable-id',
        style_table = {'height' : '27vh'},
        columns=[
            {
                "name": i,
                "id": i,
                "deletable": False,
                "selectable": True
            } for i in df.columns
        ],
        data=df.to_dict('records'),
        
        # Set up of features for interactive data table to make it user-friendly for client
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_selectable="single",
        row_deletable=False,
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=8
        
    ),
    
    html.Hr(),
    
    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex', 'justify-content':'space-around', 'align-items':'center'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
    
    
])

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    
    # create df based on the filter selection of the dropdown
    match filter_type:
        case "Water Rescue":
            df = pd.DataFrame.from_records(shelter.read({
                "animal_type":"Dog",
                "breed":{"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]},
                "sex_upon_outcome":"Intact Female",
                "age_upon_outcome_in_weeks":{"$gte":26},
                "age_upon_outcome_in_weeks":{"$lte":156}
            }))
            
        case "Mountain or Wilderness Rescue":
            df = pd.DataFrame.from_records(shelter.read({
                "animal_type":"Dog",
                "breed":{"$in":["German Shepard","Alaskan Malamute","Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
                "sex_upon_outcome":"Intact Male",
                "age_upon_outcome_in_weeks":{"$gte":26},
                "age_upon_outcome_in_weeks":{"$lte":156}
            }))
            
        case "Disaster Rescue or Individual Tracking":
            df = pd.DataFrame.from_records(shelter.read({
                "animal_type":"Dog",
                "breed":{"$in":["Doberman Pinscher","German Shepard","Golden Retriever", "Bloodhound", "Rottweiler"]},
                "sex_upon_outcome":"Intact Male",
                "age_upon_outcome_in_weeks":{"$gte":20},
                "age_upon_outcome_in_weeks":{"$lte":300}
            }))
            
        case _:
            df = pd.DataFrame.from_records(shelter.read({}))
    
    # remove the _id field from our newly constructed dataframe
    df.drop(columns=['_id'],inplace=True)
    
    # return our filtered dataframe to the table
    return df.to_dict('records')


# This callback will highlight a row on the data table when the user selects it
# for our geolocation map to update
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')]
)
def update_styles(selected_rows):
    return [{
        'if': { 'row_index': i },
        'background_color': '#D2F3FF'
    } for i in selected_rows]


# Histogram that displays how many of each animal were rescued at what age
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    
    dff = pd.DataFrame(viewData)
    
    dff['age in years'] = dff['age_upon_outcome_in_weeks'] / 52
    
    fig = px.histogram(dff, x='age in years', nbins=21, title='Distribution of Age Amongst Rescue Animals', text_auto=True)
    
    # Use integers for quantities in each age range
    fig.update_yaxes(tickformat="d")
    
    return [
        dcc.Graph(            
            figure = fig,
            style = {
                'height' : '40vh',
                'width' : '48vw'
            }
        )    
    ]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    
    # Loop until we have returned a result from our initialization of dff
    # This way on page refresh we ensure that dff is populated before
    # trying to access the lat and long coords
    while(dff.empty):
        continue
     
    # Because we only allow single row selection, the list can 
    # be converted to a row index here
    if index is None:
        row = 0
    else:
        row = index[0]

    # Austin TX is at [30.75,-97.48],
    # but I decided to center the map on the selected data point location instead
    
    # Marker with tool tip and popup
    # Column 13 and 14 define the x-y map-coordinates
    # Column 4 defines the breed for the animal
    # Column 9 defines the name of the animal
    return [
        dl.Map(
            style={'width': '48vw', 'height': '40vh'},
            center=[dff.iloc[row,13], dff.iloc[row,14]], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),                
                dl.Marker(
                    position=[dff.iloc[row,13], dff.iloc[row,14]],
                    children=[
                        dl.Tooltip(dff.iloc[row,4]),
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(dff.iloc[row,9])
                        ])
                    ]
                )
            ]
        )
    ]


# Run app and display result in jupyterlab mode, note, if you have previously run a prior app, the default port of 8050 may not be available, if so, try setting an alternate port.
app.run_server() 

Dash app running on https://joshuastore-heroiccircus-3000.codio.io/proxy/8050/
